# M4W4 Advanced topics in Python

Welocome to the last workshop of your second module of this course. Today we will cover:

- What do the undersocres mean in Python naming
- The constructor fucntion `__new__` and how is it different from `__init__`
- What are the `*args` and `**kw}args` fucntion inputs in Python
- What decorators are and why they are useful in Python

# How underscores are used in naming variables in Python

As you have noticed in the previous workshop, we have worked with the `__init__` method in clases as a *constructor*; a way to create instances of classes.

If this is your first encounter with the more advanced elements of Python, you would have noticed that except the `__init__` method, we haven't encountered any other use of the underscore, single or double, in the naming variable. 

But underscores play a very important role when naming variables within the OOP space. Underscores can come into the following four flavors:

- `_single_leading_underscore`: weak "internal use" indicator. E.g. `from M import *` does not import objects whose name starts with an underscore.


- `single_trailing_underscore_`: used by convention to avoid conflicts with Python keyword, e.g. `Tkinter.Toplevel(master, class_='ClassName')`


- `__double_leading_underscore`: when naming a class attribute, invokes name mangling (inside class FooBar, `__boo` becomes `_FooBar__boo`).


- `__double_leading_and_trailing_underscore__`: "magic" objects or attributes that live in user-controlled namespaces. E.g. `__init__`, `__import__` or `__file__`. **Never invent such names; only use them as documented**.

Let's work on some examples:

- ### `_single_leading_underscore`

In [1]:
from typing import Union

class SunderLeading():
    
    def __init__(self):
        pass
    
    def _calulate_sum(self, x:Union[int, float], y:Union[int,float]) -> Union[int,float]:
        return x + y
    
    def print_sum(self, x:Union[int, float], y:Union[int,float]) -> Union[int,float]:
        print(f"The sum of {x} and {y} is {self._calulate_sum(x,y)}")
        

### Q: What is the *correct* way of prining thre sum of two numbers using the above class?

In [4]:
SunderLeading().print_sum(2,3)

The sum of 2 and 3 is 5


The function `_calculate_sum` should **not be used** outside the class! While Python cannot enforce it, we use the single underscore `_` for that reason. If you search on the documentation of the most widely used packages in Python you see that they make use of this *very often*

- ### `single_trailing_undersocre_`

This can be used even outside the OOP score, when defining things like lists, sets etc

In [ ]:
list_: list[int] = [1,2,3]
set_: set[int] = {1,2,3}

- ### `__double_leading_underscore`

These must be used with caution becuase their use can **affect the behaviour of your classes**! Let's see the following example:

In [9]:
class DunderLeading():

    def __init__(self, bmi: float, name: str):
        self.bmi = bmi
        self.name = name
        self.__category = "Susceptible"
        if self.bmi < 5:
            self.__category = "Normal"

    def cal_category(self):
        print(self.__category)
    
    def update_category(self,new_category: str):
        self.__category = new_category

### Q: How do we update the category in this case?

In [18]:
class_ = DunderLeading(bmi=19, name="Leo")
class_._DunderLeading__category

'Susceptible'

In [21]:
[1,2]

2

**The only proper way to call a dunder attribute (with leading underscore) is to use `instance._classname__attribute`!**

- ### `__double_leadning_and_trailing_underscores`

You should **never define your own dunder fucntions**. Althought, there are some built-in useful ones that you can (and should) use of needed. For example:

- `__init__`: Instantiates a class
- `__new__`: Creates a class (in the background, but we will bring it in the foreground as well)
- `__dict__`: Creates a dictionary with all the object's attributes and/or methods.

Let's take a look in one of the previously defined classes:

In [22]:
DunderLeading.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.DunderLeading.__init__(self, bmi: float, name: str)>,
              'cal_category': <function __main__.DunderLeading.cal_category(self)>,
              'update_category': <function __main__.DunderLeading.update_category(self, new_category: str)>,
              '__dict__': <attribute '__dict__' of 'DunderLeading' objects>,
              '__weakref__': <attribute '__weakref__' of 'DunderLeading' objects>,
              '__doc__': None,
              '__annotations__': {}})

# `__init__` vs `__new__`: A new constructor in town

When you create an instance of a class, Python first calls the `__new__()` method to create the object and then calls the `__init__()` method to initialize the object’s attributes.

The `__new__()` is a method of the `object` class. It has the following structure:

`object.__new__(class, *args, **kwargs)` (more on these later today)

Let's see two example of Python's class mechanics; we define the `Person` class with the `name` attribute and create a *new instance* of the `Person` class like we have done before:


In [23]:
class Person:
    def __init__(self, name):
        self.name = name


person = Person('John')

In Python, a class is callable. When you call the class to create a new object:

`person = Person('John')`

Python will call the `__new__()` and `__init__()` methods. It’s equivalent to the following method calls:

In [24]:
person = object.__new__(Person, 'John')
person.__init__('John')

Let's check the `__dict__` attribute on this class before and after initializing:

In [25]:
person = object.__new__(Person, 'John')
print(person.__dict__)

person.__init__('John')
print(person.__dict__)

{}
{'name': 'John'}


The follwoing example illustrates the sequence which Python calls the `__new__` and `__init__` method when you create a new object by calling the class:

In [26]:
class Person:
    def __new__(cls, name):
        print(f'Creating a new {cls.__name__} object...')
        obj = object.__new__(cls)
        return obj

    def __init__(self, name):
        print(f'Initializing the person object...')
        self.name = name


person = Person('John')

Creating a new Person object...
Initializing the person object...


**In practice, you use the `__new__()` method when you want to tweak the object at the instantiated time.**

Typically, when you override the `__new__()` method, you don’t need to define the `__init__()` method because everything you can do in the `__init__()` method, you can do it in the `__new__()` method.

### Q: Let's create a `Person` class and use the `__new__`method to inject the full_name attribute to the `Person` object. It should take as an input the first and last name of the person.

In [30]:
class Person:
    def __new__(cls, first_name, last_name):
        obj = object.__new__(cls)
        
        obj.first_name = first_name
        obj.last_name = last_name
        obj.full_name = first_name + last_name
        return obj

person = Person("Leo", "S")
print(person.__dict__)

{'first_name': 'Leo', 'last_name': 'S', 'full_name': 'LeoS'}


# When the input is not that straighforward: `*args` and `**kargs`

So far, we have encountered cases where the input (whether this is for a fucntion or a class) are known and have fixed length. Let's take the following example:

In [ ]:
def sum_(x: Union[int, float], y: Union[int, float]):
    return x + y

### Q: What happens if we want to add an unkown amount of numbers?

In [38]:
def sum_(list_: list[Union[int, float]]):
    return sum(list_)

That was an easy one, but what happens when we have an unkown number of lists? Oblviously, we can extend it to a list of lists, but we can do better! Or even think about the case that we might have no lists to parse at all!

For this, and some more cases, Python has implemented two types of argumens: `*args` and `**kargs`.

We can pass a variable number of arguments to a function using special symbols. There are two special symbols:

- *args (Non Keyword Arguments)
- **kwargs (Keyword Arguments)

Let's tackle them indepenently.

- ### *args (Non Keyword Arguments)

Python has `*args` which allow us to pass the variable number of non keyword arguments to function.

In the function, we should use an asterisk `*` before the parameter name to pass variable length arguments.The arguments are passed as a tuple and these passed arguments make tuple inside the function with same name as the parameter excluding asterisk `*`.

### Q: How can we use the `*args` fucnctionality to create a fucntion that adds the numbers in an *unknown* number of lists?

In [35]:
def f(*args):
    result = 0
    for list_ in args:
        result += sum(list_)
    return result

f([1,2,3], [1,2,3])

12

In [42]:
f()

0

- ### **kargs

But what if we want to pass keyword arguments? Python is having us covered with the `**kwargs` fucntionality.

In the function, we use the double asterisk `**` before the parameter name to denote this type of argument. The arguments are passed as a dictionary and these arguments make a dictionary inside function with name same as the parameter excluding double asterisk `**`.

### Q: Let's create an example together. Let's create a function that takes a **kwargs argument as input and prints its content as pairs.

In [48]:
def f(**args):
    for key, value in args.items():
        print(key, value)

#f([1,2,3], [1,2,3])
f(Firstname= "Leo", Age=0)

Firstname Leo
Age 0


While `*args` and `**kwargs` are very handy and need minimal tuning, **they need to be used with caution**! As there is no way to use typings on the argument, there must be an internal way of checking the nature of the input (somehow, *if possible*).

# Decorators: Adding extra functionality to functions

A decorator is a design pattern in Python that allows a user to add new functionality to an existing object *without modifying its structure*. Decorators are usually called before the definition of a function you want to *decorate*.

Decorators are mostly used in the OOP world (not exclusively though). As we can have different kind of fucntions in a class, decoarators hep us to differntiate between different kinds of fucntion. Let's see some examples:

 - ### @staticmethod
 
A static method is a general utility method that performs a task in isolation. Static methods in Python are similar to those found in Java or C++. A static method **is bound to the class and not the object of the class**. Therefore, we can call it using the class name.

A static method doesn’t have access to the class and instance variables because it does not receive an implicit first argument like `self` and `cls`. Therefore it cannot modify the state of the object or class. 

Let's see an example:

In [49]:
from typing import Any

class Example:
    
    def __init__(self, attr1: Any, attr2: Any):
        self.attr1 = attr1
        self.attr2 = attr2
        
    @staticmethod
    def pritnt_useless_message():
        print("This is a useless message!")

instance = Example("bar", "foo")
instance.pritnt_useless_message()

This is a useless message!


In [3]:
class Example():
    def __new__(cls, input_: str):
        if input_.endswith("txt"):
            pass
        elif input_.endswith("json"):
            pass
        else:
            raise TypeError

- ### @classmethod

A `@classmethod` is a method inside the class that **actually creates a class**. `@classmethods` take the `cls` argument instead of the `self`. This might be counter intuitive, but lets see an example:

In [9]:
class Student(object):
    
    def __init__(self, first_name:str, last_name: str) -> None:
        self.first_name = first_name
        self.last_name = last_name
    
    @classmethod
    def from_json(cls, json) -> "Student":
        jsfile = json.read(json)
        first_name = jsfile["First Name"]
        last_name = jsfile["Last Name"]
        student = cls(first_name, last_name)
        return student
    
    @classmethod
    def from_string(cls, name_str: str) -> "Student":
        first_name, last_name = map(str, name_str.split(' '))
        student = cls(first_name, last_name)
        return student

scott = Student.from_string('Scott Robinson')
scott.first_name

'Scott'

### Q: This could be easily written using the traditional form of class creation. Can you think a case where this could be limiting in this setting?